In [22]:
import os
import OpenDartReader
import FinanceDataReader as fdr

In [23]:
stock_list = fdr.StockListing("KRX").dropna()

### openDART 한번에 모으기

In [40]:
stock_list = stock_list.loc[stock_list['Market'].isin(["KOSDAQ"])] # Market 기준 필터링
stock_name_list = stock_list.loc[stock_list['Region'].notnull(),"Name"].tolist() # 선물과 같이 KOSPI에는 있으나 배당이 없을수밖에 없는 종목은 제외
import OpenDartReader
my_api = "6f1dcbd821a73d922006b9b9efeac4eebf97df13"
dart = OpenDartReader(my_api)

In [53]:
stock_name_list


['3S',
 'APS홀딩스',
 'AP시스템',
 'AP위성',
 'BNGT',
 'CBI',
 'CJ ENM',
 'CJ 바이오사이언스',
 'CJ프레시웨이',
 'CMG제약',
 'CNH',
 'CNT85',
 'CS',
 'CSA 코스믹',
 'DMS',
 'DSC인베스트먼트',
 'EDGC',
 'EG',
 'ES큐브',
 'EV수성',
 'FSN',
 'GH신소재',
 'GST',
 'HB솔루션',
 'HB테크놀러지',
 'HK이노엔',
 'HLB',
 'HLB생명과학',
 'HLB제약',
 'HRS',
 'IBKS제12호스팩',
 'IBKS제16호스팩',
 'ISC',
 'ITX-AI',
 'JTC',
 'JW신약',
 'JYP Ent.',
 'KBG',
 'KBI메탈',
 'KB오토시스',
 'KCC건설',
 'KCI',
 'KD',
 'KG ETS',
 'KG모빌리언스',
 'KG이니시스',
 'KH 전자',
 'KH바텍',
 'KMH',
 'KMH하이텍',
 'KNN',
 'KPX생명과학',
 'KT서브마린',
 'LX세미콘',
 'NEW',
 'NE능률',
 'NHN벅스',
 'NHN한국사이버결제',
 'NICE평가정보',
 'PN풍년',
 'RFHIC',
 'RF머트리얼즈',
 'S&K폴리텍',
 'SBI인베스트먼트',
 'SBI핀테크솔루션즈',
 'SBS콘텐츠허브',
 'SBW생명과학',
 'SCI평가정보',
 'SDN',
 'SFA반도체',
 'SG',
 'SG&G',
 'SGA',
 'SGA솔루션즈',
 'SGC이테크건설',
 'SM C&C',
 'SM Life Design',
 'SNK',
 'SV인베스트먼트',
 'THE E&M',
 'THE MIDONG',
 'THQ',
 'TJ미디어',
 'TKG애강',
 'TPC',
 'TS인베스트먼트',
 'UCI',
 'WI',
 'YBM넷',
 'YTN',
 'YW',
 'iMBC',
 '가비아',
 '가온미디어',
 '감성코퍼레이션',
 '강스템바이오텍',
 '강원에너지',
 '갤럭시

In [32]:
import numpy as np
import pandas as pd
def find_financial_indicators(stock_name, year, indicators):
    report = dart.finstate(stock_name, year) # 데이터 가져오기
    if report is None: # 리포트가 없다면 (참고: 리포트가 없으면 None을 반환함)
        # 리포트가 없으면 당기, 전기, 전전기 값 모두 제거
        data = [[stock_name, year] + [np.nan] * len(indicators)]
        data = [[stock_name, year-1] + [np.nan] * len(indicators)]
        data = [[stock_name, year-2] + [np.nan] * len(indicators)]
        return pd.DataFrame(data, columns = ["기업", "연도"] + indicators)
    
    else:
        report = report[report['account_nm'].isin(indicators)] # 관련 지표로 필터링
        if sum(report['fs_nm'] == "연결재무제표") > 0:
            # 연결재무제표 데이터가 있으면 연결재무제표를 사용
            report = report.loc[report['fs_nm'] == "연결재무제표"]
            
        else:
            # 연결재무제표 데이터가 없으면 일반재무제표를 사용
            report = report.loc[report['fs_nm'] == "요약재무정보"]
        
        data = []
        for y, c in zip([year, year-1, year-2], ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']):
            record = [stock_name, y]
            for indic in indicators:
                # account_nm이 indic인 행의 c 컬럼 값을 가져옴
                if sum(report['account_nm'] == indic) > 0: # 해당 지표가 있다면 추가 (지표가 없는 경우도 있음)
                    value = report.loc[report['account_nm'] == indic, c].iloc[0]
                else:
                    value = np.nan
                
                record.append(value)
            
            data.append(record)
        return pd.DataFrame(data, columns = ["기업", "연도"] + indicators)

In [18]:
result = dart.finstate("LG전자", 2020, "11011") # 2020년 LG전자 사업보고서 내 재무 정보 가져오기
result = result.loc[result['fs_nm'] == "연결재무제표"] # 연결재무제표 필터링
# account_nm이 당기순이익, 영업이익, 매출액 중 하나인 경우에만 가져오기
result = result.loc[result['account_nm'].isin(['당기순이익', '영업이익', '매출액'])] 

# 열 필터링
result = result.loc[:, ["account_nm", "fs_nm", "sj_nm", "thstrm_dt", "thstrm_amount"]]


In [33]:
import time 
indicators = ['유동자산', '재고자산', '비유동자산','자산총계','유동부채','비유동부채', '부채총계','자본금','단기차익금','장기차익금','사채','이익잉여금','자본총계', '매출액', '영업이익', '당기순이익', '매출채권', '이자의 수취', '이자의 지급','매출총이익','유형자산','무형자산','영업비용','영업수익','순이자손익']
data = pd.DataFrame() # 이 데이터프레임에 각각의 데이터를 추가할 예정

for idx, stock_name in enumerate(stock_name_list):
    print(idx+1, "/", len(stock_name_list)) # 현재까지 진행된 상황 출력
    for year in [2017, 2020]:
        try: # 재무제표 데이터가 잘 불러와지지 않는 경우가 있어, try - except으로 넘김
            result = find_financial_indicators(stock_name, year, indicators) # 재무지표 데이터 
        except:
            pass
        data = pd.concat([data, result], axis = 0, ignore_index = True) # data에 부착
        time.sleep(0.5)

1 / 1431
2 / 1431
3 / 1431
4 / 1431
5 / 1431
6 / 1431
7 / 1431
8 / 1431
9 / 1431
10 / 1431
11 / 1431
12 / 1431
13 / 1431
14 / 1431
15 / 1431
16 / 1431
17 / 1431
18 / 1431
19 / 1431
20 / 1431
21 / 1431
22 / 1431
23 / 1431
24 / 1431
25 / 1431
26 / 1431
27 / 1431
28 / 1431
29 / 1431
30 / 1431
31 / 1431
32 / 1431
33 / 1431
34 / 1431
35 / 1431
36 / 1431
37 / 1431
38 / 1431
39 / 1431
40 / 1431
41 / 1431
42 / 1431
43 / 1431
44 / 1431
45 / 1431
46 / 1431
47 / 1431
48 / 1431
49 / 1431
50 / 1431
51 / 1431
52 / 1431
53 / 1431
54 / 1431
55 / 1431
56 / 1431
57 / 1431
58 / 1431
59 / 1431
60 / 1431
61 / 1431
62 / 1431
63 / 1431
64 / 1431
65 / 1431
66 / 1431
67 / 1431
68 / 1431
69 / 1431
70 / 1431
71 / 1431
72 / 1431
73 / 1431
74 / 1431
75 / 1431
76 / 1431
77 / 1431
78 / 1431
79 / 1431
80 / 1431
81 / 1431
82 / 1431
83 / 1431
84 / 1431
85 / 1431
86 / 1431
87 / 1431
88 / 1431
89 / 1431
90 / 1431
91 / 1431
92 / 1431
93 / 1431
94 / 1431
95 / 1431
96 / 1431
97 / 1431
98 / 1431
99 / 1431
100 / 1431
101 / 14

In [8]:
# 중복된 행 제거: 
# data.drop_duplicates(inplace = True)

In [36]:
# 숫자로 모두 변환
def str_to_float(value):
    if type(value) == float: # nan의 경우에는 문자가 아니라, 이미 float 취급됨
        return value
    elif value == '-': # -로 되어 있으면 0으로 변환
        return 0
    else:
        return float(value.replace(',', ''))

for indc in indicators:
    data[indc] = data[indc].apply(str_to_float)

###
   * 부채비율 = (총부채 / 자본총계)*100%
   * 영업이익 증가율 = (당기 영업이익 - 전기 영업이익) / 전기 영업이익 * 100%

   * 영업이익/매출액/당기순이익 상태

   * ROA = (당기순이익 / 자산총계) * 100%

   * ROE = (당기순이익 / 평균 자기 자본)

In [37]:
# 안정성
data['유동비율'] = data['유동자산'] / data['유동부채'] * 100

# data['당좌비율'] = data['당좌자산'] / data['유동부채'] * 100

data['부채비율'] = data['부채총계'] / data['자본총계'] * 100

# data['차입금의존도'] = data['장단기차입금']+data['사채'] / data['자산총계'] * 100

# data['이자보상배율] = data['장단기차입금']+data['사채'] / data['자산총계'] * 100




# 수익성
# data['매출총이익률'] = data['매출총이익'] / data['매출액'] * 100

data['영업이익률'] = data['영업이익'] / data['매출액'] * 100

data['당기순이익률'] = data['당기순이익'] / data['매출액'] * 100

data['ROA'] = data['당기순이익'] / data['자산총계']

average_equity = data['자본총계'].rolling(2).mean() # 평균 자기 자본
data['ROE'] = data['당기순이익'] / average_equity

# 활동성


# 성장성
data.sort_values(by = ['기업', '연도'], inplace = True) # 기업과 연도를 기준으로 정렬
data['매출액증가율'] = data['매출액'].diff() / data['매출액']

data.sort_values(by = ['기업', '연도'], inplace = True) # 기업과 연도를 기준으로 정렬
data['영업이익증가율'] = data['영업이익'].diff() / data['영업이익']

# data.sort_values(by = ['기업', '연도'], inplace = True) 
# data['유형자산증가율'] = data['유형자산'].diff() / data['유형자산']

data.sort_values(by = ['기업', '연도'], inplace = True) # 기업과 연도를 기준으로 정렬
data['총자산증가율'] = data['자산총계'].diff() / data['자산총계']

In [38]:
data.to_csv("./코스닥주요재무지표15.csv", index = False,encoding='CP949')